In [176]:
import cv2
import numpy as np
from cv2 import VideoWriter, VideoWriter_fourcc

In [181]:
# Create a light gray image
frame = cv2.imread(filename='white_background.jpg')
car = cv2.imread(filename='car.png')
car = cv2.resize(car,(240,300))
bus = cv2.imread(filename='bus.png')
bus = cv2.resize(bus,(220,380))
car2 = cv2.imread(filename='car2.png')
car2 = cv2.resize(car2,(180,320))
x_offset_car = 900
y_offset_car = 700
x_offset_bus = 1200
y_offset_bus = 700
x_offset_car2 = 600
y_offset_car2 = 700
frame[y_offset_car:y_offset_car+car.shape[0], x_offset_car:x_offset_car+car.shape[1]] = car
frame[y_offset_bus:y_offset_bus+bus.shape[0], x_offset_bus:x_offset_bus+bus.shape[1]] = bus
frame[y_offset_car2:y_offset_car2+car2.shape[0], x_offset_car2:x_offset_car2+car2.shape[1]] = car2
cv2.imwrite("Generated_frame.png", frame)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

True

In [182]:
# Initialize video parameters
width = 1920
height = 1080
FPS = 30
seconds = 20
initial_y_offset = 700
in_frame = True

In [ ]:
fourcc = VideoWriter_fourcc(*'MP42')
video = VideoWriter('./HUD.mp4', fourcc, float(FPS), (width, height))
for _ in range(FPS * seconds):
    frame = cv2.imread(filename='white_background.jpg')
    frame = cv2.resize(frame,(1920,1080))
    car = cv2.imread(filename='car.png')
    car = cv2.resize(car,(240,300))
    bus = cv2.imread(filename='bus.png')
    bus = cv2.resize(bus,(220,380))
    car2 = cv2.imread(filename='car2.png')
    car2 = cv2.resize(car2,(180,320))
    if (y_offset_car > car.shape[1] and y_offset_bus > bus.shape[1] and y_offset_car2 > car2.shape[1]):
        y_offset_car -= 3
        frame[y_offset_car:y_offset_car+car.shape[0], x_offset_car:x_offset_car+car.shape[1]] = car
        if (in_frame):
            y_offset_bus -= 1
            y_offset_car2 -= 2
            frame[y_offset_bus:y_offset_bus+bus.shape[0], x_offset_bus:x_offset_bus+bus.shape[1]] = bus
            frame[y_offset_car2:y_offset_car2+car2.shape[0], x_offset_car2:x_offset_car2+car2.shape[1]] = car2
    elif (y_offset_car <= car.shape[1]):
        y_offset_car = initial_y_offset
        frame = cv2.imread(filename='white_background.jpg')
        frame = cv2.resize(frame,(1920,1080))
        frame[y_offset_car:y_offset_car+car.shape[0], x_offset_car:x_offset_car+car.shape[1]] = car
        in_frame = False
    video.write(frame)
video.release()